## RAG

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['OPENAI_API_KEY']= os.getenv("OPENAI_API_KEY")


In [3]:
# VectorStoreIndex : it will convert the text into vector and also assigns the index to it
# SimpleDirectoryReader : it will read the data from the directory
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader
documents=SimpleDirectoryReader("data").load_data()

In [4]:
documents

[Document(id_='70045b78-78ee-473b-a9f6-653fcca6ac1b', embedding=None, metadata={'page_label': '1', 'file_name': '1706.03762v7.pdf', 'file_path': 'c:\\Users\\Lenovo\\Desktop\\LLAMA_INDEX_PROJECTS\\indexing_and_query\\data\\1706.03762v7.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2024-04-09', 'last_modified_date': '2024-04-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Us

In [5]:
# Converting the text into vectors
index=VectorStoreIndex.from_documents(documents,show_progress=True)

c:\Users\Lenovo\Desktop\LLAMA_INDEX_PROJECTS\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 46/46 [00:03<00:00, 14.50it/s]


In [10]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor

# SimilarityPostprocessor : find the similar result who had min 0.80 similarity score 
retriever=VectorIndexRetriever(index=index,similarity_top_k=4)
postprocessor=SimilarityPostprocessor(similarity_cutoff=0.40)

query_engine=RetrieverQueryEngine(retriever=retriever,
                                  node_postprocessors=[postprocessor])

In [11]:
responses = query_engine.query("what is Chain-of-thought Prompt.")

In [12]:
# Checking other similar responses
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(responses,show_source=True)
print(responses)

Final Response: Chain-of-thought Prompt is a method that enhances the
inference of Large Language Models (LLMs) by incorporating an
intermediate reasoning step to identify cues present in the dialogue
context. This method aims to provide more personalized and engaging
responses by prompting the models to reason based on the user status
exhibited in the conversation.
______________________________________________________________________
Source Node 1/4
Node ID: 496d2db7-4836-4a48-80d3-252b3185d2df
Similarity: 0.821114264578233
Text: to a test query sample while some works utilize uncertainty
(Diao et al., 2023) or diversity (Li and Qiu, 2023) to refine and
evaluate the selected exam- ples. Also, few works (Deng et al., 2023a)
focus on the intermediate reasoning steps, and they use the reasoning
complexity (Fu et al., 2023), i.e., chains with more reasoning steps,
making the...
______________________________________________________________________
Source Node 2/4
Node ID: 93ca4df7-20cb-4

In [14]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What are transformers?")
print(response)

Transformers are a model architecture that relies entirely on an attention mechanism to draw global dependencies between input and output sequences. They eschew recurrence and instead use self-attention to compute representations of the input and output without relying on sequence-aligned recurrent neural networks or convolutional layers.
